In [168]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
from tsetlin import TsetlinMachine
import torch
import random

DATASET_DIR = '../datasets/'
DATA_FILE = 'bit_1.txt'

text_rows = open(f'{DATASET_DIR}{DATA_FILE}', 'r').read().splitlines()
dataset = [ [int(num) for num in row.split(',')] for row in text_rows]
tensor_dataset = torch.tensor(dataset)
train_x = tensor_dataset[:, :-1]
train_y = tensor_dataset[:, -1]

# seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
# random.seed(seed)
# torch.manual_seed(seed)
# print(seed)
# random.seed(1320387042447901345)
# torch.manual_seed(1320387042447901345)
tm = TsetlinMachine(train_x.shape[1])

for i in range(6):
    shuffled_idx = torch.randperm(train_x.shape[0])
    shuffled_x = train_x[shuffled_idx]
    shuffled_y = train_y[shuffled_idx]

    for j, (x, y) in enumerate(zip(shuffled_x, shuffled_y)):
        out_1 = tm.forward(x).item()
        tm.update(y)
        out_2 = tm.forward(x).item()
        assert y.item() == out_2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [169]:
tm.l1.W,tm.l2.W[:, :tm.l2.W.shape[1]//2],tm.l2.W[:, tm.l2.W.shape[1]//2:]

(tensor([[45,  0, 45,  0, 45,  0],
         [45,  0,  0,  0, 45, 45],
         [ 0, 45, 45,  0,  0,  0],
         [45,  0,  0,  0,  0,  0],
         [ 0, 45,  0, 44,  0, 45],
         [ 0, 45, 45, 45,  0,  0],
         [45,  0,  0,  0,  0,  0],
         [ 0,  0, 45, 45, 45,  0],
         [ 0, 45,  0, 44,  0,  0],
         [ 0,  0,  0,  0, 45, 45]]),
 tensor([[ 0,  0,  0, 44,  0,  0, 43,  0,  0,  0]]),
 tensor([[ 0,  0,  0,  0, 44, 45,  0, 45, 44,  0]]))

In [170]:
for x, y in zip(train_x, train_y):
    out = tm.forward(x).item()
    assert y.item() == out

In [171]:
failed_count = 0
for _ in range(1000):
    tm = TsetlinMachine(train_x.shape[1])
    for i in range(6):
        shuffled_idx = torch.randperm(train_x.shape[0])
        shuffled_x = train_x[shuffled_idx]
        shuffled_y = train_y[shuffled_idx]

        for j, (x, y) in enumerate(zip(shuffled_x, shuffled_y)):
            out_1 = tm.forward(x).item()
            tm.update(y)
            out_2 = tm.forward(x).item()
            assert y.item() == out_2

    for x, y in zip(train_x, train_y):
        out = tm.forward(x).item()
        if y.item() != out:
            failed_count += 1
failed_count

317